<a href="https://colab.research.google.com/github/MAN1108/github-test/blob/main/mistral%207b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [4]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = None # we won't use LlamaIndex to set up LLM
Settings.chunk_size = 256
Settings.chunk_overlap = 25

LLM is explicitly disabled. Using MockLLM.


In [6]:


documents = SimpleDirectoryReader("/content/sample_data/data").load_data()

In [7]:
print(len(documents))

3917


In [8]:
index = VectorStoreIndex.from_documents(documents)

In [36]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [37]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [45]:
query = "What does Hermione say Jupiter's largest moon is?"
response = query_engine.query(query)

In [46]:
 #reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
“Listen . . . 
shall we just ask Hermione if we ca n have a look at what she’s done?” 
Harry glanced over at her; she was sitting with Crookshanks on her 
lap and chatting merrily to Ginny as a pair of knitting needles flashed 
in midair in front of her, now knitting a pair of shapeless elf socks. 
“No,” he said heavily, “you  know she won’t let us.” 
And so they worked on while the sky outside the windows became 
steadily darker; slowly, the crowd in the common  room began to thin 
again. At half-past eleven, He rmione wandered over to them, 
yawning. 
“Nearly done?” 
“No,” said Ron shortly. 
“Jupiter’s biggest moon is Gany mede, not Callisto,” she said, 
pointing over Ron’s shoulder at a line in his Astronomy essay, “and it’s 
Io that’s got the volcanos.” 
“Thanks,” snarled Ron, scratching  out the offending sentences.

"How do you expect to find a rat in the dark?" snapped Hermione.  
"There's nothing we can do! We came back to help Sirius; we're not  
supposed to be doing 

In [47]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

#config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
#model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [48]:
# prompt (no context)
intstructions_string = f"""you are a helpful question answering system \
 Answer as briefly as you can to the question. \
you are required to give one-shot answer.  \
answers end with its signature '–QnA Assistant'.

Please answer to the following question.
"""

prompt_template = lambda question: f'''[INST] {intstructions_string} \n{question} \n[/INST]'''
question = "What does Hermione say Jupiter's largest moon is?"

prompt = prompt_template(question)
print(prompt)

[INST] you are a helpful question answering system  Answer as briefly as you can to the question. you are required to give one-shot answer.  answers end with its signature '–QnA Assistant'.

Please answer to the following question.
 
What does Hermione say Jupiter's largest moon is? 
[/INST]


In [49]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                          max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] you are a helpful question answering system  Answer as briefly as you can to the question. you are required to give one-shot answer.  answers end with its signature '–QnA Assistant'.

Please answer to the following question.
 
What does Hermione say Jupiter's largest moon is? 
[/INST] Hermione in Harry Potter series mentions Ganymede as Jupiter's largest moon –QnA Assistant.</s>


In [50]:
 #prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]QnA Assistant, Answer as briefly as you can to the following question.You are reuired to give a one-shot answer. \
It  ends answers with its signature '–QnA Assistant'. \

{context}
Please respond to the following Question. Use the context above if it is helpful.

{question}
[/INST]
"""

In [51]:
prompt = prompt_template_w_context(context, question)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]QnA Assistant, Answer as briefly as you can to the following question.You are reuired to give a one-shot answer. It  ends answers with its signature '–QnA Assistant'. 
Context:
“Listen . . . 
shall we just ask Hermione if we ca n have a look at what she’s done?” 
Harry glanced over at her; she was sitting with Crookshanks on her 
lap and chatting merrily to Ginny as a pair of knitting needles flashed 
in midair in front of her, now knitting a pair of shapeless elf socks. 
“No,” he said heavily, “you  know she won’t let us.” 
And so they worked on while the sky outside the windows became 
steadily darker; slowly, the crowd in the common  room began to thin 
again. At half-past eleven, He rmione wandered over to them, 
yawning. 
“Nearly done?” 
“No,” said Ron shortly. 
“Jupiter’s biggest moon is Gany mede, not Callisto,” she said, 
pointing over Ron’s shoulder at a line in his Astronomy essay, “and it’s 
Io that’s got the volcanos.” 
“Thanks,” snarled Ron, scratching  out the o